In [3]:
import numpy as np
import sympy as sp
import pickle
import os

In [4]:
# Define the output directory
output_dir = "./thr_output"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [5]:
# Symbolically compute the expected value vector, µ_n = [S_n; R_n].

# All theory is derived in 'Revisiting the Luria_Delbrük experiment:
#                           The case of reversible non-genetic resistance'

# p, q: real numbers in [0, 1]
p = sp.Symbol('p', real=True, nonnegative=True) # probability of becoming resistant from sensitive
q = sp.Symbol('q', real=True, nonnegative=True) # probability of becoming sensitive from resistant

# n: integer
n = sp.Symbol('n', integer=True, nonnegative=True) # number of generations

# dphi(i)/ds is the expected number of sensitive offspring produced by a parent of the ith type
# dphi(i)/dt is the expected number of resistant offspring produced by a parent of the ith type
dphi1_ds = 2 * (1-p)
dphi1_dt = 2 * p
dphi2_ds = 2 * q
dphi2_dt = 2 * (1-q)

# matrix of expectations
M = sp.Matrix([
    [dphi1_ds, dphi1_dt],
    [dphi2_ds, dphi2_dt]
])

# compute the eigenvalue, eigenvector decomposition of M^T
# s.t. M^T = X * D * X^-1
X, D = (M.T).diagonalize()
X_inv = X.inv()


# initial conditions for mean, µ0_1, µ0_2: real numbers
mu_0_1 = sp.Symbol('mu_0_1', real=True, nonnegative=True)
mu_0_2 = sp.Symbol('mu_0_2', real=True, nonnegative=True)
mu_0 = sp.Matrix([[mu_0_1], [mu_0_2]])

# the forrmula for expected value is µ_n = (M^T)^n * u_0
# which can be simplified to µ_n = X * D^n * X^-1 * u_0
# for easy computation.
mu_n = sp.simplify(X * D**n * X_inv * mu_0)

# export the symbolic expression
with open(os.path.join(output_dir, 'mu_n.pkl'), 'wb') as file:
    pickle.dump(mu_n, file)
    
# to open the expression, use:
# ```
# import sympy as sp
# import pickle
# with open('mu_n.pkl', 'rb') as file:
#     mu_n = pickle.load(file)

In [6]:
mu_n

Matrix([
[(mu_0_1*(2**n*q + p*(-2*p - 2*q + 2)**n) + mu_0_2*q*(2**n - (-2*p - 2*q + 2)**n))/(p + q)],
[(mu_0_1*p*(2**n - (-2*p - 2*q + 2)**n) + mu_0_2*(2**n*p + q*(-2*p - 2*q + 2)**n))/(p + q)]])

In [7]:
# Symbolically compute the variance vector, V_n = [Var(S_n); Cov(S_n, R_n); Var(R_n)].
# All theory is derived in 'Revisiting the Luria_Delbrük experiment:
#                           The case of reversible non-genetic resistance'

# p, q: real numbers in [0, 1]
p = sp.Symbol('p', real=True, nonnegative=True) # probability of becoming resistant from sensitive
q = sp.Symbol('q', real=True, nonnegative=True) # probability of becoming sensitive from resistant

# n, k: integers
n = sp.Symbol('n', integer=True, nonnegative=True) # number of generations
k = sp.Symbol('k', integer=True, nonnegative=True) # dummy variable for n in a summation

# initial conditions for mean, µ0_1, µ0_2: real numbers
mu_0_1 = sp.Symbol('mu_0_1', real=True, nonnegative=True)
mu_0_2 = sp.Symbol('mu_0_2', real=True, nonnegative=True)
mu_0 = sp.Matrix([[mu_0_1], [mu_0_2]])

# compute the variances/covariances for the number of offpsring produced by a single parent of type i
# store these in the matrix Sigma
sigmaS_1  =  4 * p * (1-p)
sigmaSR_1 = -4 * p * (1-p)
sigmaR_1  =  4 * p * (1-p)

sigmaS_2  =  4 * q * (1-q)  
sigmaSR_2 = -4 * q * (1-q)  
sigmaR_2  =  4 * q * (1-q)

Sigma = sp.Matrix([
    [sigmaS_1,  sigmaS_2],
    [sigmaSR_1, sigmaSR_2],
    [sigmaR_1,  sigmaR_2]
])

# L matrix as defined in the theory derivations
L = sp.Matrix([
    [M[0,0]**2,               2*M[0,0]*M[1,0],               M[1,0]**2],
    [M[0,0]*M[0,1],     M[0,0]*M[1,1]+M[0,1]*M[1,0],     M[1,0]*M[1,1]],
    [M[0,1]**2,               2*M[0,1]*M[1,1],               M[1,1]**2]
])

# substitue the dummy variable k into the symbolic expression for mu_n
mu_k = mu_n.subs({n: k})

# diagonalize L to make the matrix computations (raising to power) easier
P, W  = L.diagonalize()
P_inv = P.inv()
W_inv = W.inv()

# initial conditions for variance, v0_1, v0_2, v0_3: real numbers
v_0_1 = sp.Symbol('v_0_1', real=True, nonnegative=True)
v_0_2 = sp.Symbol('v_0_2', real=True, nonnegative=True)
v_0_3 = sp.Symbol('v_0_3', real=True, nonnegative=True)
v_0 = sp.Matrix([[v_0_1], [v_0_2], [v_0_3]])

# v_n = P * W^n(P_inv*v_0 + W_inv*∑_{k=0}^{n-1}(W^{-k} * P_inv*Sigma*X * D^k) * X_inv*µ_0)
# we will define the summation term as sum_term and compute this first

# Let's define K(mu) = W^{-k} * P_inv * Sigma * X * D^k to avoid the repeated computation
K = W**(-k) * P_inv * Sigma * X * D**k

# now compute the sum term 
# since K is a 3 x 2 matrix, compute each element of the sum separately using applyfunc
# sp.summation only works with scalar expressions
sum_term = K.applyfunc(lambda term: sp.summation(term, (k, 0, n - 1)))

# use the sum_term to compute v_n using the formula for v_n shown above
v_n = P * W**n * (P_inv * v_0 + W_inv * sum_term * X_inv * mu_0)
v_n = sp.simplify(v_n)

# export the symbolic expression
with open(os.path.join(output_dir, 'v_n.pkl'), 'wb') as file:
    pickle.dump(v_n, file)
    
# to open the expression, use:
# ```
# import sympy as sp
# import pickle
# with open('v_n.pkl', 'rb') as file:
#     v_n = pickle.load(file)

In [8]:
v_n

Matrix([
[Piecewise(((4**n*q**2*(p + q)**3*(v_0_1 + 2*v_0_2 + v_0_3)*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1) + 2*q*(4*(-p - q + 1))**n*(p + q)**3*(p*v_0_1 - q*v_0_3 + v_0_2*(p - q))*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1) + (4*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1))**n*(-n*(mu_0_1*p*(p**4 + 2*p**3*q - p**3 - p**2*q - 2*p*q**3 + p*q**2 - q**4 + q**3 + q*(p**3 + 3*p**2*q - 2*p**2 + 3*p*q**2 - 4*p*q + q**3 - 2*q**2)) + mu_0_2*q*(-p**4 - 2*p**3*q + p**3 + p**2*q + 2*p*q**3 - p*q**2 + p*(p**3 + 3*p**2*q - 2*p**2 + 3*p*q**2 - 4*p*q + q**3 - 2*q**2) + q**4 - q**3))*(p**2 + 2*p*q + q**2) + (p + q)**3*(p**2*v_0_1 - 2*p*q*v_0_2 + q**2*v_0_3)*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1)))/((p + q)**3*(p**2 + 2*p*q + q**2)*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1)), Eq((p + q - 1)**(-2), 2) & Eq(1/(p + q - 1), -2)), ((1/(2*(p + q - 1)**2))**n*(q*(2*(p + q - 1)**2)**n*(p + q)**3*(4**n*q*(v_0_1 + 2*v_0_2 + v_0_3) + 2*(4*(-p - q + 1))**n*(p*v_0_1 - q*v_0_3 + v_0_2*(p - q)))*(2*(p + q - 1)**2 - 1)*(p**2 + 2*p*q - 2*p